In [32]:
import cv2
import pickle
import pandas as pd
import numpy as np
import mediapipe as mp

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [33]:
df = pd.read_csv('Pose_keypoints.csv')
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,Left,0.627484,0.151723,-0.647896,0.998232,0.648523,0.121703,-0.603666,0.995263,0.657341,...,0.239542,0.109149,0.522030,0.995119,0.145659,0.049162,0.333944,0.966942,0.124182,0.058633
1,Left,0.641352,0.151721,-0.970539,0.998358,0.663978,0.121710,-0.934947,0.995574,0.673015,...,0.790160,0.101617,0.471389,1.217810,0.692263,0.046613,0.289944,1.177155,0.611688,0.055068
2,Left,0.651244,0.153861,-1.094983,0.998514,0.674605,0.122374,-1.056288,0.995984,0.684231,...,0.873756,0.101586,0.444216,1.300892,0.849193,0.044818,0.260650,1.272532,0.639477,0.057369
3,Left,0.662199,0.154675,-1.070333,0.998633,0.684339,0.122271,-1.033326,0.996270,0.694827,...,0.856349,0.100147,0.440901,1.434736,0.744298,0.041918,0.253972,1.375614,0.615659,0.056087
4,Left,0.661861,0.154551,-0.618257,0.998767,0.684329,0.122267,-0.596261,0.996634,0.694696,...,0.573611,0.104624,0.425197,1.521169,0.526637,0.042365,0.254608,1.464597,0.467664,0.059998


In [34]:
X = df.drop('class', axis=1)
y = df['class']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
print('Train shape:')
print(X_train.shape)
print(X_test.shape)
print('\nTest Shape:')
print(y_train.shape)
print(y_test.shape)

Train shape:
(380, 132)
(96, 132)

Test Shape:
(380,)
(96,)


In [37]:
# X_train = StandardScaler().fit_transform(X_train)
# X_test = StandardScaler().fit_transform(X_test)

In [38]:
rc = RandomForestClassifier()

In [39]:
model = rc.fit(X_train, y_train)

In [40]:
y_pred = model.predict(X_test)
y_pred

array(['Nothing', 'Left', 'Right', 'Left', 'Left', 'Right', 'Up', 'Down',
       'Up', 'Left', 'Up', 'Up', 'Up', 'Nothing', 'Right', 'Nothing',
       'Left', 'Right', 'Nothing', 'Down', 'Nothing', 'Down', 'Down',
       'Nothing', 'Left', 'Nothing', 'Left', 'Right', 'Nothing', 'Down',
       'Up', 'Down', 'Nothing', 'Down', 'Left', 'Down', 'Left', 'Nothing',
       'Right', 'Left', 'Left', 'Nothing', 'Down', 'Nothing', 'Left',
       'Nothing', 'Nothing', 'Right', 'Up', 'Down', 'Up', 'Left', 'Up',
       'Down', 'Down', 'Down', 'Left', 'Nothing', 'Nothing', 'Down',
       'Right', 'Left', 'Left', 'Up', 'Right', 'Up', 'Left', 'Nothing',
       'Up', 'Nothing', 'Up', 'Nothing', 'Nothing', 'Up', 'Nothing',
       'Left', 'Nothing', 'Nothing', 'Left', 'Nothing', 'Up', 'Left',
       'Down', 'Up', 'Up', 'Right', 'Down', 'Nothing', 'Left', 'Nothing',
       'Up', 'Right', 'Left', 'Down', 'Nothing', 'Up'], dtype=object)

In [41]:
print(accuracy_score(y_test, y_pred))

1.0


In [42]:
with open('saved_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [43]:
with open('saved_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [44]:
model

RandomForestClassifier()

In [45]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [46]:
cap = cv2.VideoCapture(1)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        

        results = holistic.process(image)

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)) 

        try:
            
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            row = pose_row
            
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), 
                                                 [640,480]).astype(int))
            
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            cv2.putText(image, 'CLASS', (95,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0], (90,40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'PROB', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2)), (10,40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()